## 5 - Monitoring 

### 5.1 - Monitoring for ML-based services

**What to monitor?**

- Service health;
- Model Performance; (feedback about the model sometimes has a certain delay)
- Data quality and integrity;
- Data and Concept Drift

Extra Comprehensive monitoring:
- Performance by segment;
- Model bias/fairness;
- Outliers;
- Explainability

**Architecture ML Service**

 ![](images/2022-07-11-16-06-16.png)

For this we need to create:
1. app.py file
2. DockerFile
3. requirements.txt
4. model file (already created in previous week)

### 5.2 - Setting up the environment

- Docker-compose:

Is responsible for having everything organize per service. Such as dependencies, ports, versions, etc.



- Create a virtual environment and activate it:
`python -m venv venv && source ./venv/bin/activate`
- Update pip:
`pip install -U pip`
- Install packages:
`pip install -r requirements.txt`
- Add virtual environment to docker group:
[Follow this link instructions](https://docs.docker.com/engine/install/linux-postinstall/)
- Run docker:
`docker-compose up --build` (for linux users)

### 5.3 - Creating a prediction service and simulating traffic

Two types of monitoring: **online** and **batch** monitoring.

- We can use `send_data.py` to send the data to our system;
- Then check the `mongo db`:

```python
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27018')
db = client.get_database('prediction_service')
data_collection = db.get_collection('data')
data = list(data_collection.find())
```

In [19]:
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27018')
db = client.get_database('prediction_service')

In [20]:
db

Database(MongoClient(host=['localhost:27018'], document_class=dict, tz_aware=False, connect=True), 'prediction_service')

In [21]:
data_collection = db.get_collection('data')

In [22]:
data_collection

Collection(Database(MongoClient(host=['localhost:27018'], document_class=dict, tz_aware=False, connect=True), 'prediction_service'), 'data')

In [23]:
data = list(data_collection.find())

In [26]:
data[-1]

{'_id': ObjectId('62de78de1a9d530970ee6738'),
 'VendorID': 2,
 'lpep_pickup_datetime': '2022-01-02T11:09:01',
 'lpep_dropoff_datetime': '2022-01-02T11:14:04',
 'store_and_fwd_flag': 'N',
 'RatecodeID': 5.0,
 'PULocationID': 43,
 'DOLocationID': 75,
 'passenger_count': 1.0,
 'trip_distance': 0.82,
 'fare_amount': 11.4,
 'extra': 0.0,
 'mta_tax': 0.0,
 'tip_amount': 1.54,
 'tolls_amount': 0.0,
 'ehail_fee': None,
 'improvement_surcharge': 0.3,
 'total_amount': 13.24,
 'payment_type': 1.0,
 'trip_type': 1.0,
 'congestion_surcharge': 0.0,
 'id': 'cb154aed-b85e-48c1-ae0d-628b7bb20f99',
 'PU_DO': '43_75',
 'prediction': 7.0011621546113005}

In [25]:
len(data)

1345

- One can then stop running and check grafana

### 5.4 - Realtime monitoring walktrough (Prometheus, Evidently, Grafana)

**Grafana:**
- Enter grafana using `http://localhost:3000/?orgId=1`
- Use `admin` for both user and password

**Prometheus:**
- Enter prometheus using: `http://localhost:9091/`
- And then one can query from the data created for the dashboard.
    - Example: `evidently:data_drift:p_value`

**Metrics used for monitoring?**

### 5.5 - Batch monitoring walktrough (Prefect, MongoDB, Evidently)

- Start Prefect orion: `prefect orion start` - and check localhost created
- Run **prefect example**: `python prefect_example.py`

In [30]:
report_collection = db.get_collection('report')
report = list(report_collection.find())

In [43]:
report[0]['data_drift']

{'name': 'data_drift',
 'datetime': '2022-07-25 14:08:11.719152',
 'data': {'utility_columns': {'date': None,
   'id': None,
   'target': 'target',
   'prediction': 'prediction'},
  'cat_feature_names': ['PULocationID', 'DOLocationID'],
  'num_feature_names': ['trip_distance', 'target', 'prediction'],
  'datetime_feature_names': [],
  'target_names': None,
  'options': {'confidence': None,
   'drift_share': 0.5,
   'nbinsx': 10,
   'xbins': None},
  'metrics': {'n_features': 5,
   'n_drifted_features': 4,
   'share_drifted_features': 0.8,
   'dataset_drift': True,
   'trip_distance': {'current_small_hist': [[0.20267446893867802,
      0.0558814379378248,
      0.012510769687572715,
      0.0033362052500193915,
      0.0033362052500193915,
      0.002919179593766966,
      0.0,
      0.0008340513125048476,
      0.0004170256562524238,
      0.0004170256562524242],
     [0.0,
      3.5420000000000003,
      7.0840000000000005,
      10.626000000000001,
      14.168000000000001,
      17.

### Hands-on tutorial of how to apply evidently

- [Machine Learning Systems Design](https://www.youtube.com/watch?v=RIc45j1c520)
- [Github Evidently Repo](https://github.com/evidentlyai/evidently)